In [5]:
import os
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Function to get the start and end dates of the fiscal quarter
def get_fiscal_quarter_dates(date):
    # Convert the input datetime to a pandas Timestamp
    date = pd.Timestamp(date)
    # Get the quarter number
    quarter = date.quarter
    # Calculate the start date of the quarter
    quarter_start_date = pd.Timestamp(year=date.year, month=3 * (quarter - 1) + 1, day=1)
    # Calculate the end date of the quarter by getting the first day of the next quarter and subtracting 1 day
    quarter_end_date = (quarter_start_date + pd.offsets.QuarterEnd()).date()  # pandas automatically moves to the end of the quarter, so just add a quarter end offset
    # Format dates as strings
    return quarter_start_date.strftime('%Y-%m-%d'), quarter_end_date.strftime('%Y-%m-%d')

# Get the date from the user or any source
input_date = datetime(2021, 1, 1)

sic_code = '384'  # Modify this as needed

# Get the start and end dates of the fiscal quarter
start_date, end_date = get_fiscal_quarter_dates(input_date)

# Database connection parameters
params = {
    'host': 'public.xbrl.us',
    'port': '5432',
    'database': 'edgar_db',
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

sql_query = f"""
SELECT report.report_id, report.entity_name, report.reporting_period_end_date, report.properties->>'document_type' AS document_type
FROM report
WHERE report.reporting_period_end_date >= '{start_date}'
AND report.reporting_period_end_date <= '{end_date}'
AND (report.properties->>'standard_industrial_classification')::text LIKE '{sic_code}%'
AND report.source_id = (SELECT source_id FROM source WHERE source_name = 'SEC')
AND report.properties->>'document_type' IN ('10-Q', '10-K');
"""

# Connect to the database
try:
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()
    cursor.execute(sql_query)
    # Fetch all rows
    rows = cursor.fetchall()
    # Get column names
    col_names = [desc[0] for desc in cursor.description]
    # Create DataFrame
    df = pd.DataFrame(rows, columns=col_names)
    # Close cursor and connection
    cursor.close()
    conn.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

# Display the DataFrame
print(df)

     report_id                   entity_name reporting_period_end_date  \
0       351662          INVACARE CORPORATION                2021-03-31   
1       356694           Sanara MedTech Inc.                2021-03-31   
2       357708     BIOSIG TECHNOLOGIES, INC.                2021-03-31   
3       366315   ADM TRONICS UNLIMITED, INC.                2021-03-31   
4       343812           AngioDynamics, Inc.                2021-02-28   
..         ...                           ...                       ...   
203     357438     RESHAPE LIFESCIENCES INC.                2021-03-31   
204     353910              RenovaCare, Inc.                2021-03-31   
205     339769          Cantel Medical Corp.                2021-01-31   
206     356163          VIVEVE MEDICAL, INC.                2021-03-31   
207     332210  VARIAN MEDICAL SYSTEMS, INC.                2021-01-01   

    document_type  
0            10-Q  
1            10-Q  
2            10-Q  
3            10-K  
4          

In [6]:
# Check if all entity_name values are unique
all_unique = not df['entity_name'].duplicated().any()

if all_unique:
    print("All entity names are unique.")
else:
    print("Duplicate entity names found.")

All entity names are unique.
